In [1]:
import gym
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

In [2]:

def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

def epsilon_action(epsilon):
    random_value = np.random.random()
    if random_value<=epsilon:
        action = env.action_space.sample()
    else:
        Y=online_model.predict(state.reshape(1,-1))
        action = np.argmax(Y[0])
    return action


In [3]:


gamma=0.5
epsilon_decay = 0.005
learning_rate =0.0005
max_e=1
min_e=0
replay_batch_size=30
iterations=2000

In [4]:
env = gym.make('LunarLander-v2')
state=env.reset()
online_model = initiate_NN(learning_rate)
target_model =initiate_NN(learning_rate)
target_model.set_weights(online_model.get_weights())
memory=[]
episode_reward_rolling = []
episode_reward=[]

for episode in range(iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                target_q = target_model.predict(np.vstack(batch[:, 3]))
                y[non_terminal_samples] += np.multiply(gamma, \
                            target_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)
        if done:
            target_model.set_weights(online_model.get_weights())

    episode_reward.append(current_episode_reward) 
    if len(memory)>=3000:
        memory = memory[-3000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[-20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  0  episode reward:  -70.87  rolling reward:  -70.87  Steps:  74  epsilon:  1.00
episode:  1  episode reward:  -57.92  rolling reward:  -64.40  Steps:  62  epsilon:  1.00
episode:  2  episode reward:  -67.52  rolling reward:  -65.44  Steps:  59  epsilon:  0.99
episode:  3  episode reward:  -239.14  rolling reward:  -108.86  Steps:  104  epsilon:  0.99
episode:  4  episode reward:  -108.36  rolling reward:  -108.76  Steps:  70  epsilon:  0.98
episode:  5  episode reward:  -57.68  rolling reward:  -100.25  Steps:  84  epsilon:  0.98
episode:  6  episode reward:  -105.13  rolling reward:  -100.95  Steps:  135  epsilon:  0.97
episode:  7  episode reward:  -241.17  rolling reward:  -118.47  Steps:  69  epsilon:  0.97
episode:  8  episode reward:  -268.71  rolling reward:  -135.17  Steps:  122  epsilon:  0.96
episode:  9  episode reward:  -378.28  rolling reward:  -159.48  Steps:  106  epsilon:  0.96
episode:  10  episode reward:  -97.31  rolling reward:  -153.83  Steps:  72  epsilo

episode:  88  episode reward:  -79.78  rolling reward:  -156.44  Steps:  188  epsilon:  0.64
episode:  89  episode reward:  -121.95  rolling reward:  -156.06  Steps:  126  epsilon:  0.64
episode:  90  episode reward:  -138.10  rolling reward:  -155.86  Steps:  165  epsilon:  0.64
episode:  91  episode reward:  -186.84  rolling reward:  -156.20  Steps:  137  epsilon:  0.63
episode:  92  episode reward:  -146.02  rolling reward:  -156.09  Steps:  170  epsilon:  0.63
episode:  93  episode reward:  -147.73  rolling reward:  -156.00  Steps:  180  epsilon:  0.63
episode:  94  episode reward:  -179.04  rolling reward:  -156.24  Steps:  195  epsilon:  0.63
episode:  95  episode reward:  -244.46  rolling reward:  -157.16  Steps:  179  epsilon:  0.62
episode:  96  episode reward:  -260.71  rolling reward:  -158.23  Steps:  228  epsilon:  0.62
episode:  97  episode reward:  -67.17  rolling reward:  -157.30  Steps:  127  epsilon:  0.62
episode:  98  episode reward:  -86.49  rolling reward:  -156.5

episode:  175  episode reward:  -145.28  rolling reward:  -143.67  Steps:  517  epsilon:  0.42
episode:  176  episode reward:  -102.08  rolling reward:  -143.78  Steps:  178  epsilon:  0.41
episode:  177  episode reward:  -32.24  rolling reward:  -143.59  Steps:  157  epsilon:  0.41
episode:  178  episode reward:  -183.19  rolling reward:  -144.97  Steps:  179  epsilon:  0.41
episode:  179  episode reward:  -51.45  rolling reward:  -143.64  Steps:  189  epsilon:  0.41
episode:  180  episode reward:  -65.12  rolling reward:  -143.03  Steps:  220  epsilon:  0.41
episode:  181  episode reward:  -186.96  rolling reward:  -142.87  Steps:  234  epsilon:  0.40
episode:  182  episode reward:  -229.95  rolling reward:  -143.70  Steps:  293  epsilon:  0.40
episode:  183  episode reward:  -58.55  rolling reward:  -143.78  Steps:  204  epsilon:  0.40
episode:  184  episode reward:  -102.19  rolling reward:  -142.82  Steps:  269  epsilon:  0.40
episode:  185  episode reward:  -113.19  rolling rewar

KeyboardInterrupt: 

In [5]:

for episode in range(207,iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                target_q = target_model.predict(np.vstack(batch[:, 3]))
                y[non_terminal_samples] += np.multiply(gamma, \
                            target_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)
        if done:
            target_model.set_weights(online_model.get_weights())

    episode_reward.append(current_episode_reward) 
    if len(memory)>=3000:
        memory = memory[-3000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[-20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  207  episode reward:  -325.94  rolling reward:  -138.45  Steps:  314  epsilon:  0.36
episode:  208  episode reward:  -142.11  rolling reward:  -137.05  Steps:  409  epsilon:  0.35
episode:  209  episode reward:  -91.94  rolling reward:  -136.73  Steps:  421  epsilon:  0.35
episode:  210  episode reward:  -280.88  rolling reward:  -137.74  Steps:  580  epsilon:  0.35
episode:  211  episode reward:  -29.78  rolling reward:  -136.99  Steps:  306  epsilon:  0.35
episode:  212  episode reward:  -10.69  rolling reward:  -135.39  Steps:  117  epsilon:  0.35
episode:  213  episode reward:  -172.66  rolling reward:  -136.44  Steps:  85  epsilon:  0.34
episode:  214  episode reward:  -295.72  rolling reward:  -137.43  Steps:  194  epsilon:  0.34
episode:  215  episode reward:  -35.09  rolling reward:  -136.00  Steps:  456  epsilon:  0.34
episode:  216  episode reward:  -124.15  rolling reward:  -135.10  Steps:  445  epsilon:  0.34
episode:  217  episode reward:  -35.45  rolling reward:

KeyboardInterrupt: 

In [6]:

for episode in range(230,iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                target_q = target_model.predict(np.vstack(batch[:, 3]))
                y[non_terminal_samples] += np.multiply(gamma, \
                            target_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)
        if done:
            target_model.set_weights(online_model.get_weights())

    episode_reward.append(current_episode_reward) 
    if len(memory)>=3000:
        memory = memory[-3000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[-20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  230  episode reward:  -309.90  rolling reward:  -135.11  Steps:  170  epsilon:  0.32
episode:  231  episode reward:  -29.16  rolling reward:  -134.23  Steps:  372  epsilon:  0.32
episode:  232  episode reward:  -36.45  rolling reward:  -133.50  Steps:  198  epsilon:  0.31
episode:  233  episode reward:  -258.58  rolling reward:  -135.08  Steps:  163  epsilon:  0.31
episode:  234  episode reward:  -153.82  rolling reward:  -135.17  Steps:  146  epsilon:  0.31
episode:  235  episode reward:  77.85  rolling reward:  -133.78  Steps:  1000  epsilon:  0.31
episode:  236  episode reward:  -232.28  rolling reward:  -135.75  Steps:  179  epsilon:  0.31
episode:  237  episode reward:  -16.79  rolling reward:  -133.25  Steps:  209  epsilon:  0.31
episode:  238  episode reward:  -52.95  rolling reward:  -133.18  Steps:  541  epsilon:  0.30
episode:  239  episode reward:  -128.52  rolling reward:  -133.39  Steps:  588  epsilon:  0.30
episode:  240  episode reward:  -155.88  rolling reward

episode:  317  episode reward:  -97.09  rolling reward:  -137.19  Steps:  196  epsilon:  0.20
episode:  318  episode reward:  40.32  rolling reward:  -135.46  Steps:  1000  epsilon:  0.20
episode:  319  episode reward:  -230.31  rolling reward:  -135.20  Steps:  221  epsilon:  0.20
episode:  320  episode reward:  -27.48  rolling reward:  -132.10  Steps:  268  epsilon:  0.20
episode:  321  episode reward:  69.95  rolling reward:  -131.19  Steps:  1000  epsilon:  0.20
episode:  322  episode reward:  -282.35  rolling reward:  -133.54  Steps:  249  epsilon:  0.20
episode:  323  episode reward:  138.37  rolling reward:  -130.84  Steps:  1000  epsilon:  0.20
episode:  324  episode reward:  93.97  rolling reward:  -128.59  Steps:  1000  epsilon:  0.20
episode:  325  episode reward:  -616.65  rolling reward:  -130.69  Steps:  156  epsilon:  0.20
episode:  326  episode reward:  -24.62  rolling reward:  -128.58  Steps:  180  epsilon:  0.20
episode:  327  episode reward:  -88.45  rolling reward: 

episode:  404  episode reward:  -17.41  rolling reward:  -100.36  Steps:  317  epsilon:  0.13
episode:  405  episode reward:  -203.92  rolling reward:  -101.94  Steps:  330  epsilon:  0.13
episode:  406  episode reward:  -211.62  rolling reward:  -101.25  Steps:  297  epsilon:  0.13
episode:  407  episode reward:  -80.18  rolling reward:  -100.98  Steps:  232  epsilon:  0.13
episode:  408  episode reward:  -177.38  rolling reward:  -102.26  Steps:  224  epsilon:  0.13
episode:  409  episode reward:  -263.62  rolling reward:  -104.67  Steps:  439  epsilon:  0.13
episode:  410  episode reward:  -448.80  rolling reward:  -109.13  Steps:  307  epsilon:  0.13
episode:  411  episode reward:  -352.90  rolling reward:  -112.07  Steps:  273  epsilon:  0.13
episode:  412  episode reward:  -286.75  rolling reward:  -111.48  Steps:  489  epsilon:  0.13
episode:  413  episode reward:  24.08  rolling reward:  -110.89  Steps:  267  epsilon:  0.13
episode:  414  episode reward:  4.09  rolling reward: 

episode:  491  episode reward:  257.39  rolling reward:  -100.73  Steps:  437  epsilon:  0.09
episode:  492  episode reward:  -11.10  rolling reward:  -100.49  Steps:  356  epsilon:  0.09
episode:  493  episode reward:  -44.59  rolling reward:  -97.75  Steps:  254  epsilon:  0.09
episode:  494  episode reward:  -207.71  rolling reward:  -98.42  Steps:  411  epsilon:  0.08
episode:  495  episode reward:  -59.14  rolling reward:  -94.31  Steps:  293  epsilon:  0.08
episode:  496  episode reward:  -285.88  rolling reward:  -94.15  Steps:  257  epsilon:  0.08
episode:  497  episode reward:  244.55  rolling reward:  -87.63  Steps:  425  epsilon:  0.08
episode:  498  episode reward:  -232.38  rolling reward:  -88.42  Steps:  239  epsilon:  0.08
episode:  499  episode reward:  -201.37  rolling reward:  -90.26  Steps:  227  epsilon:  0.08
episode:  500  episode reward:  -20.25  rolling reward:  -89.54  Steps:  203  epsilon:  0.08
episode:  501  episode reward:  -237.18  rolling reward:  -91.80

episode:  579  episode reward:  -83.48  rolling reward:  -52.28  Steps:  247  epsilon:  0.06
episode:  580  episode reward:  119.11  rolling reward:  -49.43  Steps:  989  epsilon:  0.06
episode:  581  episode reward:  -219.18  rolling reward:  -49.86  Steps:  300  epsilon:  0.05
episode:  582  episode reward:  -266.69  rolling reward:  -51.85  Steps:  279  epsilon:  0.05
episode:  583  episode reward:  -41.18  rolling reward:  -49.66  Steps:  249  epsilon:  0.05
episode:  584  episode reward:  -106.68  rolling reward:  -48.11  Steps:  238  epsilon:  0.05
episode:  585  episode reward:  -17.35  rolling reward:  -50.90  Steps:  316  epsilon:  0.05
episode:  586  episode reward:  28.80  rolling reward:  -51.35  Steps:  246  epsilon:  0.05
episode:  587  episode reward:  -195.80  rolling reward:  -52.88  Steps:  232  epsilon:  0.05
episode:  588  episode reward:  -170.41  rolling reward:  -52.72  Steps:  1000  epsilon:  0.05
episode:  589  episode reward:  -188.78  rolling reward:  -54.41 

...

episode:  755  episode reward:  21.61  rolling reward:  -112.53  Steps:  233  epsilon:  0.02
episode:  756  episode reward:  -226.52  rolling reward:  -116.58  Steps:  320  epsilon:  0.02
episode:  757  episode reward:  -38.01  rolling reward:  -116.01  Steps:  238  epsilon:  0.02
episode:  758  episode reward:  -265.04  rolling reward:  -116.77  Steps:  727  epsilon:  0.02
episode:  759  episode reward:  -202.43  rolling reward:  -118.83  Steps:  246  epsilon:  0.02
episode:  760  episode reward:  167.93  rolling reward:  -119.63  Steps:  452  epsilon:  0.02
episode:  761  episode reward:  -66.75  rolling reward:  -120.09  Steps:  277  epsilon:  0.02
episode:  762  episode reward:  -65.57  rolling reward:  -118.16  Steps:  254  epsilon:  0.02
episode:  763  episode reward:  -1.01  rolling reward:  -114.57  Steps:  251  epsilon:  0.02
episode:  764  episode reward:  -119.12  rolling reward:  -113.26  Steps:  366  epsilon:  0.02
episode:  765  episode reward:  -77.90  rolling reward:  -

episode:  842  episode reward:  -182.62  rolling reward:  -93.62  Steps:  310  epsilon:  0.01
episode:  843  episode reward:  11.57  rolling reward:  -91.60  Steps:  293  epsilon:  0.01
episode:  844  episode reward:  138.37  rolling reward:  -87.75  Steps:  654  epsilon:  0.01
episode:  845  episode reward:  -124.24  rolling reward:  -86.84  Steps:  887  epsilon:  0.01
episode:  846  episode reward:  -313.83  rolling reward:  -89.34  Steps:  774  epsilon:  0.01
episode:  847  episode reward:  -182.42  rolling reward:  -89.69  Steps:  340  epsilon:  0.01
episode:  848  episode reward:  -28.13  rolling reward:  -88.06  Steps:  225  epsilon:  0.01
episode:  849  episode reward:  14.27  rolling reward:  -87.78  Steps:  1000  epsilon:  0.01
episode:  850  episode reward:  -187.93  rolling reward:  -89.25  Steps:  257  epsilon:  0.01
episode:  851  episode reward:  -201.48  rolling reward:  -89.21  Steps:  303  epsilon:  0.01
episode:  852  episode reward:  19.83  rolling reward:  -87.73  S

episode:  930  episode reward:  -182.85  rolling reward:  -40.45  Steps:  313  epsilon:  0.01
episode:  931  episode reward:  -16.79  rolling reward:  -38.37  Steps:  285  epsilon:  0.01
episode:  932  episode reward:  207.08  rolling reward:  -36.79  Steps:  386  epsilon:  0.01
episode:  933  episode reward:  -191.25  rolling reward:  -41.21  Steps:  297  epsilon:  0.01
episode:  934  episode reward:  -237.53  rolling reward:  -46.28  Steps:  347  epsilon:  0.01
episode:  935  episode reward:  -19.43  rolling reward:  -46.53  Steps:  281  epsilon:  0.01
episode:  936  episode reward:  -18.10  rolling reward:  -46.19  Steps:  257  epsilon:  0.01
episode:  937  episode reward:  -174.21  rolling reward:  -47.61  Steps:  249  epsilon:  0.01
episode:  938  episode reward:  -203.49  rolling reward:  -46.47  Steps:  269  epsilon:  0.01
episode:  939  episode reward:  -112.92  rolling reward:  -50.07  Steps:  224  epsilon:  0.01
episode:  940  episode reward:  -158.50  rolling reward:  -54.69

episode:  1018  episode reward:  -64.89  rolling reward:  -4.34  Steps:  288  epsilon:  0.01
episode:  1019  episode reward:  -180.62  rolling reward:  -5.89  Steps:  241  epsilon:  0.01
episode:  1020  episode reward:  -52.30  rolling reward:  -6.02  Steps:  302  epsilon:  0.01
episode:  1021  episode reward:  236.16  rolling reward:  -5.60  Steps:  342  epsilon:  0.01
episode:  1022  episode reward:  -64.64  rolling reward:  -5.64  Steps:  326  epsilon:  0.01
episode:  1023  episode reward:  -9.62  rolling reward:  -7.98  Steps:  336  epsilon:  0.01
episode:  1024  episode reward:  258.02  rolling reward:  -3.70  Steps:  385  epsilon:  0.01
episode:  1025  episode reward:  222.02  rolling reward:  -3.45  Steps:  409  epsilon:  0.01
episode:  1026  episode reward:  -222.37  rolling reward:  -5.88  Steps:  346  epsilon:  0.01
episode:  1027  episode reward:  -191.10  rolling reward:  -7.66  Steps:  222  epsilon:  0.01
episode:  1028  episode reward:  -350.29  rolling reward:  -9.34  St

KeyboardInterrupt: 

In [12]:
'weights_DQN_'+str(max_e)+'-'+str(0.0001)+'-'+str(epsilon_decay)+'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5'

'weights_DQN_1-0.0001-0.005-0.5-30-0.0005.h5'

In [11]:
online_model.save_weights('weights_DQN_'+str(max_e)+'-'+str(0.0001)+'-'+str(epsilon_decay)+'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5')

import pickle

with open('weights_DQN_'+str(max_e)+'-'+str(0.0001)+'-'+str(epsilon_decay)+'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)


In [9]:
'weights_DQN_'+str(max_e)+'-'+str(0.0001)+'-'+str(epsilon_decay)+'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json'

'weights_DQN_1-0.0001-0.005-0.5-30-0.0005.json'

In [10]:
with open('weights_DQN_'+str(max_e)+'-'+str(0.0001)+'-'+str(epsilon_decay)+'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)
